# Analysis Ready Sentinel-1 Backscatter Imagery

In [1]:
import datetime
import itertools

import fsspec
import intake
import pandas as pd
import s3fs
import xarray as xr

In [2]:
# set up a connection with credentials and other settings
fs = fsspec.filesystem('s3', anon=True)
bucket = 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW'
fs.ls(bucket)

['sentinel-s1-rtc-indigo/tiles/RTC/1/IW/10',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/11',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/12',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/15',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/16',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/17',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/18',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/19']

- Documentation: https://sentinel-s1-rtc-indigo-docs.s3-us-west-2.amazonaws.com/data_format.html#data-structure

In [3]:
# set up configurations for our parameters of interest
UTM_ZONE_COMPONENT = [13, 14]
LATITUDE_BANDS = ["T", "U"]
GRID_SQUARE_IDS = ["BE", "PN"]
YEARS = [2021]
SATELLITES = ["S1B"]

In [4]:
# Loop over a combination of possible configurations and construct s3 object paths
params = list(
    itertools.product(*[UTM_ZONE_COMPONENT, LATITUDE_BANDS, GRID_SQUARE_IDS, YEARS, SATELLITES])
)
paths = [
    f"s3://{bucket}/{utm_zone}/{latitude_band}/{grid_square_id}/{year}/{satellite}*/Gamma0_VV.tif"
    for utm_zone, latitude_band, grid_square_id, year, satellite in params
]
paths

['s3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2021/S1B*/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/PN/2021/S1B*/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/U/BE/2021/S1B*/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/U/PN/2021/S1B*/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/BE/2021/S1B*/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/T/PN/2021/S1B*/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/U/BE/2021/S1B*/Gamma0_VV.tif',
 's3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/14/U/PN/2021/S1B*/Gamma0_VV.tif']

In [5]:
fs.ls('s3://sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE')

['sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2016',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2017',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2018',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2019',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2020',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2021']

In [6]:
# Look at the items from the first path
items = fs.glob(paths[0])
items

['sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2021/S1B_20210106_13TBE_ASC/Gamma0_VV.tif',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2021/S1B_20210111_13TBE_ASC/Gamma0_VV.tif',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2021/S1B_20210118_13TBE_ASC/Gamma0_VV.tif',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2021/S1B_20210123_13TBE_ASC/Gamma0_VV.tif',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2021/S1B_20210130_13TBE_ASC/Gamma0_VV.tif',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2021/S1B_20210204_13TBE_ASC/Gamma0_VV.tif',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2021/S1B_20210211_13TBE_ASC/Gamma0_VV.tif',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2021/S1B_20210223_13TBE_ASC/Gamma0_VV.tif',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2021/S1B_20210228_13TBE_ASC/Gamma0_VV.tif',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2021/S1B_20210307_13TBE_ASC/Gamma0_VV.tif',
 'sentinel-s1-rtc-indigo/tiles/RTC/1/IW/13/T/BE/2021/S1B_20210319_13TBE_ASC/Gamm

In [7]:
# Open one image
ds = xr.open_rasterio(f"s3://{items[0]}", chunks={})
ds

<xarray.DataArray (band: 1, y: 5490, x: 5490)>
dask.array<open_rasterio-f5094e40b34389bd95bcd337374547e2<this-array>, shape=(1, 5490, 5490), dtype=float32, chunksize=(1, 5490, 5490), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 4.5e+06 4.5e+06 4.5e+06 ... 4.39e+06 4.39e+06 4.39e+06
  * x        (x) float64 2e+05 2e+05 2e+05 ... 3.097e+05 3.098e+05 3.098e+05
Attributes: (12/22)
    transform:              (20.0, 0.0, 199980.0, 0.0, -20.0, 4500000.0)
    crs:                    +init=epsg:32613
    res:                    (20.0, 20.0)
    is_tiled:               1
    nodatavals:             (0.0,)
    scales:                 (1.0,)
    ...                     ...
    SCENE_1_METADATA:       {"title": "S1B_IW_GRDH_1SDV_20210106T010145_20210...
    SCENE_1_PRODUCT_INFO:   {"id": "S1B_IW_GRDH_1SDV_20210106T010145_20210106...
    SCENE_2_METADATA:       {"title": "S1B_IW_GRDH_1SDV_20210106T010210_20210...
    SCENE_2_PRODUCT_INFO:   {"id": "S1B_IW_GRDH_1SDV_20210106T010210_20210106...
    TILE_ID:                13TBE
    VALID_PIXEL_PERCENT:    54.024

In [8]:
# Function for cleaning the data: rename band -> time and create datetime object
def preprocess(ds):
    ds["band"] = [datetime.datetime.fromisoformat(ds.attrs["DATE"])]
    ds = ds.rename({'band': 'time'})
    return ds

In [9]:
preprocess(ds)

<xarray.DataArray (time: 1, y: 5490, x: 5490)>
dask.array<open_rasterio-f5094e40b34389bd95bcd337374547e2<this-array>, shape=(1, 5490, 5490), dtype=float32, chunksize=(1, 5490, 5490), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2021-01-06
  * y        (y) float64 4.5e+06 4.5e+06 4.5e+06 ... 4.39e+06 4.39e+06 4.39e+06
  * x        (x) float64 2e+05 2e+05 2e+05 ... 3.097e+05 3.098e+05 3.098e+05
Attributes: (12/22)
    transform:              (20.0, 0.0, 199980.0, 0.0, -20.0, 4500000.0)
    crs:                    +init=epsg:32613
    res:                    (20.0, 20.0)
    is_tiled:               1
    nodatavals:             (0.0,)
    scales:                 (1.0,)
    ...                     ...
    SCENE_1_METADATA:       {"title": "S1B_IW_GRDH_1SDV_20210106T010145_20210...
    SCENE_1_PRODUCT_INFO:   {"id": "S1B_IW_GRDH_1SDV_20210106T010145_20210106...
    SCENE_2_METADATA:       {"title": "S1B_IW_GRDH_1SDV_20210106T010210_20210...
    SCENE_2_PRODUCT_INFO:   {"id": "S1B_IW_GRDH_1SDV_20210106T010210_20210106...
    TILE_ID:                13TBE
    VALID_PIXEL_PERCENT:    54.024

In [10]:
dataarrays = [
    preprocess(
        xr.open_rasterio(
            f"s3://{url}",
            chunks={},
        )
    )
    for url in items
]
da = xr.concat(dataarrays, dim='time', join='override', combine_attrs='drop')
da

<xarray.DataArray (time: 16, y: 5490, x: 5490)>
dask.array<concatenate, shape=(16, 5490, 5490), dtype=float32, chunksize=(1, 5490, 5490), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2021-01-06 2021-01-11 ... 2021-04-17
  * y        (y) float64 4.5e+06 4.5e+06 4.5e+06 ... 4.39e+06 4.39e+06 4.39e+06
  * x        (x) float64 2e+05 2e+05 2e+05 ... 3.097e+05 3.098e+05 3.098e+05